Import packages

In [13]:
import pandas as pd 
import numpy as np
import itertools
import collections
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
import nltk
import re
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')


tweet_tokenizer = TweetTokenizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\findi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\findi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\findi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\findi\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [14]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
df=pd.read_csv("df_comments_all.csv")
df.head()

,Unnamed: 0,subreddit,post_title,post_url,comment_id,parent_id,comment,comment_score,comment_timestamp,is_top_level,IOS,classification_result,classification_score,adjusted_classification_score
0,0,ios,IOS 17 HERE!!,https://www.reddit.com/r/ios/comments/16m12vy/...,k15fixv,16m12vy,Is it the same build as RC,166,2023-09-18 13:08:19,True,17,NEGATIVE,0.993840,-0.993840
1,1,ios,IOS 17 HERE!!,https://www.reddit.com/r/ios/comments/16m12vy/...,k15fnph,k15fixv,same,51,2023-09-18 13:09:06,False,17,NEGATIVE,0.984588,-0.984588
2,2,ios,IOS 17 HERE!!,https://www.reddit.com/r/ios/comments/16m12vy/...,k18irgf,k15fixv,Whats Rc,1,2023-09-19 01:45:55,False,17,POSITIVE,0.877058,0.877058
3,3,ios,IOS 17 HERE!!,https://www.reddit.com/r/ios/comments/16m12vy/...,k18n8kl,k15fixv,We want root!,1,2023-09-19 02:38:34,False,17,POSITIVE,0.966101,0.966101
4,4,ios,IOS 17 HERE!!,https://www.reddit.com/r/ios/comments/16m12vy/...,k19bzvi,k15fixv,What is RC?,1,2023-09-19 07:41:07,False,17,NEGATIVE,0.996428,-0.996428


Remove Special Characters ,set all to lowercase,Tokenize and remove stopwords

In [ ]:
pattern = r'[^\w\s]'
df["comment_cleaned"]=df["comment"].str.lower()
df["comment_cleaned"] = df["comment_cleaned"].apply(lambda x: re.sub(pattern, '', x) if(pd.notnull(x)) else x )
df["unigrams"] = df["comment_cleaned"].apply(lambda x: re.sub(pattern, '', x))
df["unigrams"]=df["unigrams"].apply(tweet_tokenizer.tokenize) 

In [ ]:
def cleanCount(df,ios,sentiment):
    stop = stopwords.words('english')
    cleaned_stopwords=df[(df["IOS"]==ios )& (df["classification_result"]==sentiment)]["unigrams"].apply(lambda x: [item for item in x if item not in stop])
    all_words_nsw = list(itertools.chain(*cleaned_stopwords)) #This will generate all tokenized words into a single list #
    counts_nsw = collections.Counter(all_words_nsw) # This will create a count of all tokenized words
    final_counts_nsw=counts_nsw.most_common() # this will sort on most common words
    dfCounts=pd.DataFrame(final_counts_nsw,columns=["words","count"])
    dfCounts["ios"]=ios
    dfCounts["Sentiment"]=sentiment
    return dfCounts

Create wordcount Table

In [21]:
classificationType=list(df["classification_result"].unique())
iosType=list(df["IOS"].unique())
countDFFinal=pd.DataFrame(columns=["words","count","ios","Sentiment"])
for x in classificationType:
    for y in iosType:
        countDFFinal=pd.concat([countDFFinal,cleanCount(df,y,x)])

countDFFinal

,words,count,ios,Sentiment
0,ios,186,17,NEGATIVE
1,update,141,17,NEGATIVE
2,iphone,122,17,NEGATIVE
3,like,121,17,NEGATIVE
4,battery,105,17,NEGATIVE
...,...,...,...,...
448,long,1,15,POSITIVE
449,time,1,15,POSITIVE
450,top,1,15,POSITIVE
451,instead,1,15,POSITIVE


In [22]:
countDFFinal.to_csv('SentimentAnalysis_WordCount.csv')